In [1]:
#===============================================================
#   Program:    cn_data
#   Programmer: Daryl Ryan Chong
#   Last Modified: October 23, 2021
#
#   Purpose: This programme contains a function that returns a Data Frame of the 
#         number of Cn bright states and coupled basis for 
#         a specified range of number of atoms.
#==========================a=====================================

from itertools import product
from itertools import combinations

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sympy import (symbols, factor, sqrt, simplify, 
                   evaluate, Matrix, Rational, N, latex)
from sympy.physics.quantum import qapply, Bra, Ket
from sympy.physics.quantum import OrthogonalBra, OrthogonalKet
from sympy.physics.quantum.dagger import Dagger
from sympy import init_printing
init_printing(use_latex=True)

def cn_data(initial,final):
    #---------------------------------------
    # Input range of number of atoms
    #---------------------------------------
    n_range = list(range(initial, final))
    
    data = {'id': [],
            'number_of_atoms': [],
            'coupled_basis_vector': [],
            'normalised_eigenvectors_eigenvalues': [],
            'number_of_coupled_basis':  [],
            'number_of_bright_states': []
    }
    
    for number_of_atoms in n_range:
        # Cn
        # % is the modulo operator. It is used because the numbering of the atoms starts from 0 instead of 1.
        edges = [ (x%number_of_atoms, (x+1)%number_of_atoms) for x in range(number_of_atoms)]

        #---------------------------------------
        # Creating the Rydberg basis
        #---------------------------------------

        #creating full 2^N basis 
        basis = list(product([0,1], repeat=number_of_atoms))
        # print('basis (list of tuples): \n', basis)

        # finding the basis which have 2 excited atoms within the blockade radius
        impossible_states = []
        for e in edges: # e is e.g. (1,2)
            for i in basis: # i is e.g. (1,1,0,0)
                if i[e[0]] == 1 and i[e[1]] == 1:
                    impossible_states.append(i)

        # removing the basis which have 2 excited atoms within the blockade radius
        bright_states = [item for item in basis if item not in impossible_states]

        # separate basis according to number of excited atoms
        number_of_excited_atoms_dict = {}
        for i in bright_states:    
            number_of_excited_atoms = i.count(1)
            if number_of_excited_atoms in number_of_excited_atoms_dict:
                number_of_excited_atoms_dict[number_of_excited_atoms].append(i)
            else:
                number_of_excited_atoms_dict[number_of_excited_atoms] = [i]   

        #--------------------------------------------------
        # Separating the basis based on geometry
        #--------------------------------------------------

        for number_of_excited_atoms in number_of_excited_atoms_dict:
            sym_base_state = {} # a sym_base_state is a specific geometry based on the distance between excited atoms 

            for basis in number_of_excited_atoms_dict[number_of_excited_atoms]:
                indices = [i for i, x in enumerate(basis) if x == 1]    
                min_dist_list = [] # list of min distances between adjacent excited atoms. It is used to identify the geometry

                # finding the min distance between every pair of adjacent excited atoms:
                for i in range(number_of_excited_atoms):
                    pair_index1 = indices[i]
                    # the % operator is to change number_of_excited_atoms to 0
                    pair_index2 = indices[(i+1)%number_of_excited_atoms] 

                    # The sum of the 2 distances must sum to the length of the path around the atom, 
                    # which is equal to the number of atoms 
                    # Thus, distance2 = len - distance1
                    pair_distance = [ abs(pair_index2-pair_index1), number_of_atoms - abs(pair_index2-pair_index1) ] 
                    min_dist_list.append(min(pair_distance))
            #         print(basis, indices, pair_distance, min_dist_list)
                min_dist_list = tuple(sorted(min_dist_list)) # need to use tuple as dict key cannot be list

                if min_dist_list in sym_base_state:
                    sym_base_state[min_dist_list].append(basis)
                else: 
                    sym_base_state[min_dist_list] = [basis]

            new_basis_list = [] # to replace elements of number_of_excited_atoms_dict
            for i in sym_base_state:
                new_basis_list.append(sym_base_state[i])

            number_of_excited_atoms_dict[number_of_excited_atoms] = new_basis_list

        #--------------------------------------------------
        # Creating the coupled basis    
        #--------------------------------------------------
        coupled_basis = []

        for n in number_of_excited_atoms_dict:
            for W_basis in number_of_excited_atoms_dict[n]:
                W_state = 0
                for base in W_basis: # base is the original basis
                    ket = OrthogonalKet(*base) #transform to ket
                    W_state = W_state + ket
                W_state = W_state/(sqrt(len(W_basis))) #normalising the coupled states 
                coupled_basis.append(W_state)

        number_of_coupled_basis = (len(coupled_basis))
        
        #--------------------------------------------------
        # Constructing the Hamiltonian 
        #--------------------------------------------------

        # creating H equation
        transition_terms = 0 # first term in Hamiltonian
        # creating the symbols to be used in the equations
        omega , hbar, U = symbols('Omega,hbar,U') 

        # addittion of transition of n to n+1 excitation states and vice versa    

        max_number_of_excited_atoms = len(number_of_excited_atoms_dict) - 1
        max_number_of_excited_atoms

        for n in range(max_number_of_excited_atoms):
            for nW_basis in number_of_excited_atoms_dict[n]: # each list in number_of_excited_atoms_dict[1] represents a W state
                for n_basis in nW_basis: # n_basis is a tuple e.g. (1,0,0,0) 
                    for nplus1W_basis in number_of_excited_atoms_dict[n+1]:
                        for nplus1_basis in nplus1W_basis: # nplus1_basis is a tuple e.g. (0,0,0,0)
                            
                            # * is to unpack the elements in the iterable so that the label of the ket is 0001 instead of (0,0,0,1)
                            ket = OrthogonalKet(*n_basis)
                            bra = OrthogonalBra(*nplus1_basis)

                            # finding the amount of numbers that differ in the labels of ket and bra
                            label_similarity = [x == y for x,y in zip(n_basis,nplus1_basis)] # list containing False and True elements

                            # only if the ket and bra differ in exactly 1 of the atom's state and all other atoms are in the same state,
                            # can a transition occur. 

                            # add transition terms only if 1 atom's state is different in the ket and bra
                            if label_similarity.count(False) == 1:
                                zero_to_one = ket*bra
                                one_to_zero = Dagger(zero_to_one)
                                transition_terms = transition_terms + omega*hbar*(zero_to_one + one_to_zero)/2

        # addition of interaction terms between excited atoms
        # excitation_terms = 0
        # for i in number_of_excited_atoms_dict[2]:
        #     interaction = OrthogonalKet(*i)*OrthogonalBra(*i)
        #     excitation_terms = excitation_terms + U*interaction

        # H_terms = transition_terms + excitation_terms
        H_terms = transition_terms

        #--------------------------------------------------
        # Computing H matrix elements
        #--------------------------------------------------

        H_list = []

        for i in range(len(coupled_basis)):
            list_i = []
            for j in range(len(coupled_basis)):
                list_i.append(qapply(Dagger(coupled_basis[i])*H_terms*coupled_basis[j]))
            H_list.append(list_i)

        H_matrix = Matrix(H_list)
    
        # if U = 0,
        H_matrix = H_matrix.subs(U,0)
        # using omega = hbar = 1 units
        H_matrix = H_matrix.subs(omega,1)
        H_matrix = H_matrix.subs(hbar,1)

        
        #--------------------------------------------------
        # Calculate the eigenvectors
        #--------------------------------------------------
        eigen_list = H_matrix.eigenvects()

        # calculate normalisation factors
        norm_factor_list = []
        for i in range(len(eigen_list)):
            sum_of_components_squared = 0
            for j in range(len(eigen_list[i][2][0])):
                sum_of_components_squared  = sum_of_components_squared  + (eigen_list[i][2][0][j])**2
            norm_factor = 1/sqrt(sum_of_components_squared)
            norm_factor_list.append(norm_factor)
            
        # calculate normalised eigenvectors
        normalised_eigen_list = []
        for i in range(len(eigen_list)):
            eigenvalue = eigen_list[i][0]
            eigenvector = norm_factor_list[i]*eigen_list[i][2][0]
            val_vec = [eigenvalue,eigenvector] 
            normalised_eigen_list.append(val_vec) 
            
        coupled_basis_vector = latex(Matrix(coupled_basis))
        
        # convert complex analytical expression into a decimal value
        for i in range(len(normalised_eigen_list)):
            normalised_eigen_list[i][1] = N(normalised_eigen_list[i][1], 10)
            normalised_eigen_list[i][0] = N(normalised_eigen_list[i][0], 10)
        
        # finding the number of bright states
        number_of_bright_states = 0

        for i in range(len(normalised_eigen_list)):
            # if the coefficient of W0 is not 0, add 1 to number_of_bright_states
            if normalised_eigen_list[i][1][0] != 0:
                number_of_bright_states = number_of_bright_states + 1
        
        ID = "C"+str(number_of_atoms)
        
        data['id'].append(ID)
        data['number_of_atoms'].append(number_of_atoms)
        data['coupled_basis_vector'].append(coupled_basis_vector)
        data['normalised_eigenvectors_eigenvalues'].append(normalised_eigen_list)
        data['number_of_coupled_basis'].append(number_of_coupled_basis)
        data['number_of_bright_states'].append(number_of_bright_states)
        
    df = pd.DataFrame(data)
    return(df)

In [2]:
df = cn_data(1,6)
df

,id,number_of_atoms,coupled_basis_vector,normalised_eigenvectors_eigenvalues,number_of_coupled_basis,number_of_bright_states
0,C1,1,\left[\begin{matrix}{\left|0\right\rangle }\en...,"[[0, [1.000000000]]]",1,1
1,C2,2,\left[\begin{matrix}{\left|00\right\rangle }\\...,"[[-0.7071067812, [-0.7071067812, 0.7071067812]...",2,2
2,C3,3,\left[\begin{matrix}{\left|000\right\rangle }\...,"[[-0.8660254038, [-0.7071067812, 0.7071067812]...",2,2
3,C4,4,\left[\begin{matrix}{\left|0000\right\rangle }...,"[[0, [-0.5773502692, 0, 0.8164965809]], [-1.22...",3,3
4,C5,5,\left[\begin{matrix}{\left|00000\right\rangle ...,"[[-1.500000000, [0.5270462767, -0.7071067812, ...",3,3


In [3]:
# df.to_csv('Cn_data.csv')